In [19]:
from util.visualise import *

In [20]:
pd.options.display.float_format = '{:,.2f}'.format

In [21]:
PAPER_DIR = str(Path.home())+"/papers/24-aaai-goose/paper"

PLOT_DIR = f"{PAPER_DIR}/plots"
PLOT_DIR

'/home/dillon/papers/24-aaai-goose/paper/plots'

In [22]:
L=4
aggr="max"

CONFIGS = ["blind", "hff", "hff-pwl", "shgn", 
           "ddg-el dd", 
           "ddg-el di", 
           "sdg-el dd", "fdg-el dd", "ldg-el dd", "sdg-el di", "fdg-el di", "ldg-el di"]
LINE_STYLES = [CONFIG_TO_LINE_STYLE[config] for config in CONFIGS]
COLOURS = [CONFIG_TO_COLOUR[config] for config in CONFIGS]

In [23]:
df = collect_test_stats_planner_and_graphs(CONFIGS, L, aggr, normalise=False)

In [24]:
# convert back to map then to latex
solved = []
df_solved = df.drop(columns=["expanded", "cost", "time"])
for domain in GOOSE_DOMAINS:
  df_domain_coverage = (df_solved.loc[df['domain'] == domain]).drop(columns=["domain"])
  df_domain_coverage = df_domain_coverage.groupby(['config'])['solved'].sum()
  df_list = df_domain_coverage.to_numpy()

  tmp = {}
  for i, config in enumerate(sorted(CONFIGS)):
    tmp[config] = df_list[i]
  solved_domain = []
  for config in CONFIGS:
    solved_domain.append(tmp[config])
  solved.append(solved_domain)

In [25]:
solved

[[0, 19, 10, 0, 15, 0, 10, 11, 29, 14, 14, 0],
 [0, 90, 37, 0, 4, 0, 33, 33, 78, 15, 6, 0],
 [1, 18, 14, 0, 3, 6, 5, 9, 18, 4, 5, 9],
 [0, 36, 11, 0, 4, 0, 10, 10, 1, 3, 3, 0],
 [74, 90, 46, 0, 12, 0, 52, 56, 34, 48, 42, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 55, 0, 0, 10],
 [0, 6, 8, 0, 41, 15, 52, 35, 39, 44, 24, 37],
 [3, 26, 0, 0, 73, 0, 78, 23, 3, 37, 14, 0]]

In [26]:
ret  = r"\begin{tabularx}{\columnwidth}{p{\domainwidth} "
for _ in range( len(CONFIGS)):
    ret += "p{\coveragewidth} "
ret += r"} \toprule" + '\n'

BASELINES = len(CONFIGS) - 6
ret += r"& \multicolumn{" + str(BASELINES) + r"}{c}{baseline} & \multicolumn{3}{c}{domain-dep.} & \multicolumn{3}{c}{domain-ind.} \\" 
ret += f"\cmidrule(lr){{2-{2+BASELINES-1}}} \cmidrule(lr){{{2+BASELINES}-{2+BASELINES+2}}} \cmidrule(lr){{{2+BASELINES+3}-{2+BASELINES+5}}} " + '\n'

ret += "& "
for config in CONFIGS:
    ret += '\\header{' + CONFIG_TO_TEX[config] + '} & '
ret = ret[0:len(ret) - 2]
ret += r"\\ \midrule" + '\n'



for i, domain in enumerate(GOOSE_DOMAINS):
    ret += domain + f" ({len(os.listdir(f'../benchmarks/goose/{domain}/test'))}) & "
    ordered = sorted(solved[i], reverse=True)
    for count in solved[i]:
        if count == 0:
            LIME = ZERO
        elif count==ordered[0]:
            LIME = FIRST
        elif count==ordered[1]:
            LIME = SECOND
        elif count==ordered[2]:
            LIME = THIRD
        else:
            LIME = CELL
        ret += f"{LIME}{{{count}}}" + " & "
    ret = ret[0:len(ret) - 2]
    ret += r"\\" + '\n'


ret += r"\bottomrule\end{tabularx}"

f = open(f"{PAPER_DIR}/tables/coverage.tex", 'w')
f.write(ret)
f.close()

In [27]:
df

,config,solved,expanded,time,cost,domain
0,blind,0,-1,-1.00,-1,blocks
1,hff,0,-1,-1.00,-1,blocks
2,hff-pwl,0,-1,-1.00,-1,blocks
3,shgn,0,-1,-1.00,-1,blocks
4,ddg-el dd,0,-1,-1.00,-1,blocks
...,...,...,...,...,...,...
7291,fdg-el dd,1,26779,106.07,1117,visitsome
7292,ldg-el dd,0,-1,-1.00,-1,visitsome
7293,sdg-el di,1,1078,5.13,281,visitsome
7294,fdg-el di,1,92791,381.86,759,visitsome
